In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom

from tqdm import tqdm_notebook as tqdm

from seaborn import set_style
set_style("whitegrid")

## Opis problema:
U posudi imamo $m$ crvenih i $n$ bijelih kuglica. Izvlačimo kuglice iz posude dokle got izvlačimo istu boju. Kada izvučemo kuglicu druge boje prekidamo niz. Bacamo pristran novčić, čija je vjerojatnost da padne glava $p$, onliko puta koliko smo kuglica iste boje izvukli. Konačan rezultat je broj glava koji smo dobili.

_primjer:_
izvlačimo niz kuglica $b, b, b, b, b, c$, bacamo novčić 5 puta $p, g, g, p, g$.
Dakle rezultat je 3.

In [ ]:
# za potrebe provjere
m, n = 80, 20
p = .5
numberOfSimulations = 10**6

## Prvo simuliramo

In [ ]:
# bijelu označimo s 1
# crvenu s 0
def singleSimulation(m, n, p):
    draw = np.random.binomial(1, n / (m + n))
    counter = 1
    while True:
        #print("draw: ", draw)
        if draw:
            if n <= 1:
                break
            n -= 1
        else:

            if m <= 1:
                break
            m -= 1
        
        newDraw = np.random.binomial(1, n / (m + n))
        #print("m, n = ", m, n)
        if draw != newDraw:
            break
        else:
            draw = newDraw
            counter += 1
    heads = np.random.binomial(counter, p)
    #print("counter: ", counter)
    return heads, counter

In [ ]:
def simulation(m, n, p, simNumber):
    sims = np.empty(simNumber)
    for i in tqdm(range(simNumber)):
        sims[i], _ = singleSimulation(m, n, p)
    unique, counts = np.unique(sims, return_counts=True)
    fig, ax = plt.subplots(figsize = (15, 8))
    ax.bar(unique, counts / simNumber)
    ax.set_xlim([-1, np.round((m + n) / 4)])
    plt.show(fig)
    plt.close(fig)
    return unique, counts / simNumber

In [ ]:
simValue, simProb = simulation(m, n, p, numberOfSimulations)

## Sada rješavamo analitički

Ako sam prvo izvukao crvenu kuglicu, vjerojatnost da ću u idućem izvlačenju dobiti crvenu je $\frac{m - 1}{(m - 1) + n}$, a vjerojatnost da ću u trećem dobiti crvenu je $\frac{m - 2}{(m - 2) + n}$.
Dakle vjerojatnost da će niz izvučenih crvenih kuglica biti $k$ (gdje je $k < m$) je $$ \frac{m}{m + n} \cdot \frac{m - 1}{(m - 1) + n}
 \cdot \frac{m - 2}{(m - 2) + n} \cdot \dots \cdot \frac{(m - (k - 1))}{(m - (k - 1)) + n} \cdot \frac{n}{(m - k) + n}
 = \prod_{i = 0}^{k - 1} \frac{m - i}{(m - i) + n} \cdot \frac{n}{(m - k) + n} .$$
Zadnji faktor je tu zato što zadnja kuglica koju izvlačimo mora biti bijela.
Analogno ako smo prvo izvukli bijelu.
Dakle vjerojatnost da će niz izvučenih bijelih kuglica biti $k$ (za $k < n$) je
$$ \frac{n}{m + n} \cdot \frac{n - 1}{m + (n - 1)} \cdot \frac{n - 2}{m + (n - 2)} \cdot \dots \cdot \frac{(n - (k - 1))}{m + (n - (k - 1))} \cdot \frac{m}{m + (n - k)} = \prod_{i = 0}^{k - 1} \frac{m - i}{m + (n - i)} \cdot \frac{n}{m + (n - k)} .$$
Vjerojatnost da ćemo dobiti niz duljine $k$ je sada:
$$ \bigg( \prod_{i = 0}^{k - 1} \frac{m - i}{(m - i) + n} \bigg) \cdot \frac{n}{(m - k) + n} \cdot \mathbb{1}_{\{ k \leq m \}} 
 + \bigg( \prod_{i = 0}^{k - 1} \frac{n - i}{m + (n - i)} \bigg) \cdot \frac{m}{m + (n - k)} \cdot \mathbb{1}_{\{ k \leq n \}} $$

In [ ]:
def drawDensity(m, n, k):
    assert m + n > 0
    assert k >= 0
    
    if k == 0:
        return 0.
    
    # ako imamo samo bijele (m = 0), onda ćemo izvlačiti dok ih sve ne izvučemo
    # isto vrijedi i ako imamo samo crvene (n = 0)
    if m == 0:
        if k == n:
            return 1.
        else:
            return 0.
    if n == 0:
        if k == m:
            return 1.
        else:
            return 0.
    
    if m <= k:
        redProbability = 0
    else:
        redProbability = 1
        for i in range(k):
            redProbability *= ((m - i) / ((m - i) + n))
        redProbability *= (n / ((m - k) + n))
        
    if n <= k:
        whiteProbability = 0
    else:
        whiteProbability = 1
        for i in range(k):
            whiteProbability *= ((n - 1) / (m + (n - i)))
        whiteProbability *= (m / (m + (n - k)))
        
    return redProbability + whiteProbability

Usporedimo analitički dobivenu gustoću sa rezulatatom simulacije

In [ ]:
def drawDensityPlot(m, n):
    density = np.empty(m + n)
    for k in range(m + n):
        density[k] = drawDensity(m, n, k + 1)
    drawValue = np.arange(m + n) + 1
    fig, ax = plt.subplots(figsize = (15, 8))
    ax.bar(drawValue, density)
    ax.set_xlim([0, np.round((m + n) / 3)])
    plt.show(fig)
    plt.close(fig)
    return drawValue, density

In [ ]:
_ = drawDensityPlot(m, n)

In [ ]:
def drawDensitySimulation(m, n, simNumber):
    drawSims = np.empty(simNumber)
    for i in tqdm(range(simNumber)):
        _, drawSims[i] = singleSimulation(m, n, .5)
    unique, counts = np.unique(drawSims, return_counts=True)
    fig, ax = plt.subplots(figsize = (15, 8))
    ax.bar(unique, counts / simNumber)
    ax.set_xlim([0, np.round((m + n) / 3)])
    plt.show(fig)
    plt.close(fig)
    return unique, counts / simNumber

In [ ]:
_ = drawDensitySimulation(m, n, numberOfSimulations)

Označimo li slučajnu varijablu koja nam daje duljinu niza sa $X$, sada smo za dane parametre $m, \: n$ odredili $f_X$.
Ako slučajnu varijablu koja broji glave označimo sa $Y$, onda vrijedi:
$$ \mathbb{P}\big(Y = k \big) = \sum_{i = k}^{m \lor n} \mathbb{P}\big(X = i\big) \cdot \binom{i}{k} p^k (1 - p)^{i - k}. $$
Odnosno,
$$ \mathbb{P}\big(Y = j \big) = \sum_{k = j}^{m \lor n} \bigg( \bigg( \prod_{i = 0}^{k - 1} \frac{m - i}{(m - i) + n} \bigg) \cdot \frac{n}{(m - k) + n} \cdot \mathbb{1}_{\{ k \leq m \}} 
 + \bigg( \prod_{i = 0}^{k - 1} \frac{n - i}{m + (n - i)} \bigg) \cdot \frac{m}{m + (n - k)} \cdot \mathbb{1}_{\{ k \leq n \}} \bigg)  \cdot \binom{k}{j} p^k (1 - p)^{k - j}. $$

In [ ]:
def coinDensity(m, n, p, k):
    assert m + n > 0
    assert k >= 0
    
    maxIndex = np.max((m, n))
    
    if k > maxIndex:
        return 0.
    
    coinProbability = 0.
    for i in range(k, maxIndex + 1):
        coinProbability += drawDensity(m, n, i) * binom.pmf(k, i, p)
    return coinProbability

In [ ]:
def plotCoinDensity(m, n, p):
    density = np.empty(m + n)
    for k in range(m + n):
        density[k] = coinDensity(m, n, p, k)
    drawValue = np.arange(m + n)
    fig, ax = plt.subplots(figsize = (15, 8))
    ax.bar(drawValue, density)
    ax.set_xlim([-1, np.round((m + n) / 3)])
    plt.show(fig)
    plt.close(fig)
    return drawValue, density

In [ ]:
anValue, anProb = plotCoinDensity(m, n, p)

In [ ]:
fig, ax = plt.subplots(figsize = (15, 8))
ax.bar(simValue, simProb, color = "blue", alpha = .5, label = "simulirane vrijednost")
ax.bar(anValue, anProb, color = "red", alpha = .5, label = "analitički dobivene vrijednost")
ax.set_xlim([-1, 30])
ax.legend()
plt.show(fig)

Analitički dobivene vrijednosti imaju puno dulji rep od simulacije, to vjerojatno objašnjava precijenjene vjerojatnosti za $0$ i $1$.